*This notebooks includes code for all data requests*

## Stream Flow Data (National Water Model)

In [ ]:
!pip install nwm

In [9]:
from nwm import NwmHs

# download streamflow data
nwm_data = NwmHs()
dataset = nwm_data.get_data(archive='harvey', config='short_range', geom='channel_rt',
                            variable='streamflow', comid=[23990053], init_time=0, # change comid [] as per our selected location
                            start_date='2017-08-23') # change date as per our analysis

Get comid[] from https://hs-apps.hydroshare.org/apps/nwm-forecasts/ 

<img src="../images/comid_NWM.PNG" width=300>


In [10]:
dataset

<xarray.DataArray (time: 18)>
array([5.6504, 5.2972, 4.2378, 3.5315, 2.8252, 2.1189, 1.7657, 1.0594,
       1.0594, 0.7063, 0.7063, 0.7063, 0.7063, 0.7063, 0.7063, 0.7063,
       0.7063, 0.7063])
Coordinates:
  * time     (time) datetime64[ns] 2017-08-23T01:00:00 ... 2017-08-23T18:00:00
Attributes:
    site_name:           COMID: 23990053; Short Range, Channel (streamflow).
    variable_name:       Flow Forecast
    variable_unit_name:  Flow
    variable_unit:       cfs
    value_type:          Derived Value
    no_data_value:       -9999
    archive:             harvey

## Daily Temperature and Precipitation Data (Daymet)

In [4]:
import xarray as xr
from pyproj import CRS
import geopandas as gpd

Use xarray to open the dataset at this OPeNDAP endpoint


In [2]:
ds = xr.open_dataset( "https://thredds.daac.ornl.gov/thredds-daymet/dodsC/daymet-v3-agg/na.ncml" )

In [3]:
ds

<xarray.Dataset>
Dimensions:                  (nv: 2, time: 14600, x: 7814, y: 8075)
Coordinates:
  * x                        (x) float32 -4560250.0 -4559250.0 ... 3252750.0
  * y                        (y) float32 4984000.0 4983000.0 ... -3090000.0
  * time                     (time) datetime64[ns] 1980-01-01 ... 2019-12-31
    lat                      (y, x) float32 ...
    lon                      (y, x) float32 ...
Dimensions without coordinates: nv
Data variables:
    lambert_conformal_conic  int16 ...
    yearday                  (time) int16 ...
    time_bnds                (time, nv) datetime64[ns] ...
    dayl                     (time, y, x) float32 ...
    prcp                     (time, y, x) float32 ...
    srad                     (time, y, x) float32 ...
    swe                      (time, y, x) float32 ...
    tmax                     (time, y, x) float32 ...
    tmin                     (time, y, x) float32 ...
    vp                       (time, y, x) float32 ...
Attributes:
    start_year:        1980
    source:            Daymet Software Version 3.0
    Version_software:  Daymet Software Version 3.0
    Version_data:      Daymet Data Version 3.0
    Conventions:       CF-1.6
    citation:          Please see http://daymet.ornl.gov/ for current Daymet ...
    references:        Please see http://daymet.ornl.gov/ for current informa...
    title:             Daymet: Daily Surface Weather Data on a 1-km Grid for ...
    institution:       Oak Ridge National Laboratory Distributed Active Archi...
    end_year:          2019

**Subset data by bounding box**

In [6]:
# Specify the region we want to look at with DayMet (lon_min, lat_min, lon_max, lat_max)
# and the coordinate system we are providing the bounding box coordinates in (epsg:4326)
min_long = 
min_lt = 
max_long = 
max_lat = 
bounding_box = gpd.GeoSeries(geometry.box(min_long, min_lat, max_long, max_lat), 
                             crs='epsg:4326')

In [ ]:
# Get the coordinate reference system used by DayMet
daymet_crs = CRS.from_cf(ds.lambert_conformal_conic.attrs)

In [ ]:
# Reproject our bounding box polygon into this new coordinate system (Lambert Conformal Conic)
bounding_box = bounding_box.to_crs(daymet_crs)

In [ ]:
# Select this region from our DayMet Dataset
city_clim = ds.sel(x = slice(bounding_box.bounds.minx[0], bounding_box.bounds.maxx[0]), # Here we are slicing the x axis with slice(minx, maxx)
                   y = slice(bounding_box.bounds.maxy[0], bounding_box.bounds.miny[0])) # Note that we are slicing the *inverted* y axis with slice(maxy,miny)

# Look at what we selected from DayMet
city_clim